In [7]:
import pandas as pd
import pickle
import os

# covisit

In [2]:
pred_df = pickle.load(open("../output/cv/pred_df.pkl", "rb"))
pred_df = pred_df.explode("labels")
pred_df = pred_df.rename(columns={"labels": "aid"})
pred_df.to_csv("pred_df_test.csv", index=False)

In [20]:
df = pd.read_csv("../output/splendid-flower-28/cv/full_columns_validation_preds.csv")
df["session"] = df["session_type"].apply(lambda x: x.split("_")[0])
df["type"] = df["session_type"].apply(lambda x: x.split("_")[1])
df = df[["session", "type", "top_n"]]
df["top_n"] = df["top_n"].apply(lambda x: [int(v) for v in x.split(" ")])
df = df.explode("top_n")
df = df.rename(columns={"top_n": "aid"})
df.head()

,session,type,aid
0,11098528,clicks,11830
0,11098528,clicks,588923
0,11098528,clicks,1732105
0,11098528,clicks,571762
0,11098528,clicks,884502


In [21]:
df.shape

(100348921, 3)

In [22]:
df.to_csv("covisit.csv", index=False)

In [5]:
top_n_clicks = pickle.load(open(os.path.join("../output/splendid-flower-28/cv", f"top_20_clicks_0.pkl"), "rb"))

In [11]:
top_n_clicks[0]

[532042,
 643097,
 1848174,
 1735605,
 1363081,
 1012453,
 251581,
 1256426,
 994505,
 150507,
 39551,
 66456,
 215649,
 285817,
 319593,
 353304,
 435875,
 472412,
 505086,
 878565]

In [1]:
import pandas as pd
import gc
import pickle
import glob
from lightgbm.sklearn import LGBMRanker


def read_files(path):
    dfs = []
    for file in glob.glob(path):
        df = pd.read_parquet(file)
        df["aid"] = df["aid"].astype(int)
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)


def read_train_labels():
    train_labels = pd.read_parquet("../input/otto-validation/test_labels.parquet")
    train_labels = train_labels.explode("ground_truth")
    train_labels["aid"] = train_labels["ground_truth"]
    train_labels = train_labels[["session", "type", "aid"]]
    train_labels["aid"] = train_labels["aid"].astype(int)
    return train_labels


def dump_pickle(path, o):
    with open(path, "wb") as f:
        pickle.dump(o, f)


In [2]:
train_all = read_files("../input/lgbm_dataset/*")
print("read train files")
train_labels_all = read_train_labels()

read train files


In [3]:
train_all.shape

(49740239, 28)

In [6]:
train_all["session_length"][:30]

0     30
1     28
2     28
3     28
4     28
5     28
6     28
7     28
8     28
9     28
10    28
11    28
12    28
13    28
14    28
15    28
16    28
17    28
18    28
19    28
20    28
21    28
22    28
23    28
24    28
25    28
26    28
27    28
28    28
29    29
Name: session_length, dtype: int64

In [5]:
type = "clicks"

In [6]:
print(f"type={type} start")
train_labels = train_labels_all[train_labels_all["type"] == type]
train_labels["gt"] = 1
train = train_all.merge(train_labels, how="left", on=["session", "aid"])
del train_labels
gc.collect()
train["gt"].fillna(0, inplace=True)
train["gt"] = train["gt"].astype(int)
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=20,
    importance_type="gain",
)
target = "gt"
feature_cols = train.drop(columns=[target, "session", "type"]).columns.tolist()
# train
print(f"train shape: {train.shape}")
print(f"group sum: {train['session_length'].sum()}")

type=clicks start


/tmp/ipykernel_52/1721257436.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_labels["gt"] = 1


train shape: (49740239, 30)
group sum: 1445094293


In [7]:
train.head()

,session,aid,session_interaction_length,session_length,clicks_cnt,carts_cnt,orders_cnt,this_aid_clicks_cnt,this_aid_carts_cnt,this_aid_orders_cnt,...,min_log_recency_score,max_log_recency_score,avg_type_weighted_log_recency_score,min_type_weighted_log_recency_score,max_type_weighted_log_recency_score,covisit_clicks_candidate_num,covisit_carts_candidate_num,covisit_orders_candidate_num,type,gt
0,12245144,1791715,3,30,1,2,0,0.0,1.0,0.0,...,0.464086,0.464086,2.784514,2.784514,2.784514,NaN,NaN,NaN,NaN,0
1,12245145,29735,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14.0,9.0,17.0,NaN,0
2,12245145,137514,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.0,13.0,NaN,0
3,12245145,177958,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.0,4.0,19.0,clicks,1
4,12245145,368495,6,28,6,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.0,15.0,15.0,NaN,0


In [9]:
train[train["session"] == 12245144].shape

(30, 30)

In [10]:
train = train.sort_values(["session", "aid"])

In [11]:
train.head()

,session,aid,session_interaction_length,session_length,clicks_cnt,carts_cnt,orders_cnt,this_aid_clicks_cnt,this_aid_carts_cnt,this_aid_orders_cnt,...,min_log_recency_score,max_log_recency_score,avg_type_weighted_log_recency_score,min_type_weighted_log_recency_score,max_type_weighted_log_recency_score,covisit_clicks_candidate_num,covisit_carts_candidate_num,covisit_orders_candidate_num,type,gt
7771759,11098528,11830,1,21,1,0,0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0
7771760,11098528,77440,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,2.0,3.0,NaN,0
7771761,11098528,205357,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,0
7771762,11098528,231487,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,14.0,15.0,NaN,0
7771763,11098528,307904,1,21,1,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,14.0,1.0,2.0,NaN,0


In [ ]:
ranker = ranker.fit(
    train[feature_cols],
    train[target],
    group=train["session_length"].tolist(),
)

In [26]:
test_predictions_clicks = pickle.load(open("../output/lgbm/test_predictions_clicks.pkl", "rb"))
test_predictions_carts = pickle.load(open("../output/lgbm/test_predictions_carts.pkl", "rb"))
test_predictions_orders = pickle.load(open("../output/lgbm/test_predictions_orders.pkl", "rb"))
test_predictions_clicks["session_type"] = test_predictions_clicks["session"].apply(lambda x: str(x) + "_clicks")
test_predictions_carts["session_type"] = test_predictions_carts["session"].apply(lambda x: str(x) + "_carts")
test_predictions_orders["session_type"] = test_predictions_orders["session"].apply(lambda x: str(x) + "_orders")
sub = pd.concat([test_predictions_clicks, test_predictions_carts, test_predictions_orders])
sub["labels"] = sub["aid"].apply(lambda x: " ".join(map(str, x)))
sub[["session_type", "labels"]].to_csv("submission.csv", index=False)

In [27]:
sub

,session,aid,session_type,labels
0,12899779,"[1660089, 1660529, 1667087, 1854910, 45290, 15...",12899779_clicks,1660089 1660529 1667087 1854910 45290 159257 3...
1,12899780,"[231487, 409620, 595830, 487136, 618310, 63610...",12899780_clicks,231487 409620 595830 487136 618310 636101 7605...
2,12899781,"[3542, 374037, 528496, 754412, 759436, 811084,...",12899781_clicks,3542 374037 528496 754412 759436 811084 950718...
3,12899782,"[1033148, 229748, 479970, 1696036, 595994, 829...",12899782_clicks,1033148 229748 479970 1696036 595994 829180 16...
4,12899783,"[1567973, 1627943, 1654062, 1787028, 84297, 16...",12899783_clicks,1567973 1627943 1654062 1787028 84297 169050 2...
...,...,...,...,...
1671798,14571577,"[1349213, 1368814, 1395485, 1768884, 768043, 9...",14571577_orders,1349213 1368814 1395485 1768884 768043 935830 ...
1671799,14571578,"[1264437, 1290293, 1580943, 1662628, 372644, 4...",14571578_orders,1264437 1290293 1580943 1662628 372644 476318 ...
1671800,14571579,"[1445637, 1633220, 1680826, 1769779, 876416, 5...",14571579_orders,1445637 1633220 1680826 1769779 876416 51363 2...
1671801,14571580,"[1117970, 1339930, 1627186, 1647562, 387358, 5...",14571580_orders,1117970 1339930 1627186 1647562 387358 528665 ...


In [28]:
test_predictions_clicks = pickle.load(open("../output/lgbm/test_predictions_clicks.pkl", "rb"))
test_predictions_carts = pickle.load(open("../output/lgbm/test_predictions_carts.pkl", "rb"))
test_predictions_orders = pickle.load(open("../output/lgbm/test_predictions_orders.pkl", "rb"))
test_predictions_clicks["session_type"] = test_predictions_clicks["session"].apply(lambda x: str(x) + "_clicks")
test_predictions_carts["session_type"] = test_predictions_carts["session"].apply(lambda x: str(x) + "_carts")
test_predictions_orders["session_type"] = test_predictions_orders["session"].apply(lambda x: str(x) + "_orders")
sub = pd.concat([test_predictions_clicks, test_predictions_carts, test_predictions_orders])
sub["labels"] = sub["aid"].apply(lambda x: " ".join(map(str, x)))
sub[["session_type", "labels"]].to_csv("submission.csv", index=False)

In [29]:
sub

,session,aid,session_type,labels
0,12899779,"[1660089, 1660529, 637538, 1667087, 45290, 159...",12899779_clicks,1660089 1660529 637538 1667087 45290 159257 39...
1,12899780,"[409620, 231487, 595830, 618310, 1383529, 6361...",12899780_clicks,409620 231487 595830 618310 1383529 636101 760...
2,12899781,"[1008624, 3542, 1272792, 528496, 374037, 95071...",12899781_clicks,1008624 3542 1272792 528496 374037 950718 8110...
3,12899782,"[1033148, 229748, 479970, 1696036, 829180, 595...",12899782_clicks,1033148 229748 479970 1696036 829180 595994 16...
4,12899783,"[1500659, 84297, 697336, 354698, 294573, 15679...",12899783_clicks,1500659 84297 697336 354698 294573 1567973 169...
...,...,...,...,...
1671798,14571577,"[1395485, 493115, 1302088, 1349213, 768043, 11...",14571577_orders,1395485 493115 1302088 1349213 768043 1190776 ...
1671799,14571578,"[1264437, 98564, 5573, 1580943, 476318, 372644...",14571578_orders,1264437 98564 5573 1580943 476318 372644 83006...
1671800,14571579,"[1445637, 1633220, 1680826, 1769779, 876416, 7...",14571579_orders,1445637 1633220 1680826 1769779 876416 785544 ...
1671801,14571580,"[1647562, 1117970, 826148, 1627186, 528665, 38...",14571580_orders,1647562 1117970 826148 1627186 528665 387358 8...


In [30]:
import pandas as pd

In [31]:
pred = pickle.load(open("../output/word2vec/predictions.pkl", "rb"))

In [32]:
pred

,session,labels
0,12899779,"[59625, 987932, 854602, 1036258, 1724952, 3573..."
1,12899780,"[1142000, 736515, 973453, 582732, 1419849, 582..."
2,12899781,"[918667, 199008, 194067, 57315, 141736, 583140..."
3,12899782,"[834354, 595994, 740494, 889671, 987399, 77947..."
4,12899783,"[1817895, 607638, 1754419, 1216820, 1729553, 3..."
...,...,...
1671798,14571577,"[1141710, 768043, 983945, 361202, 1291495, 138..."
1671799,14571578,"[519105, 822641, 326442, 1572043, 1064975, 533..."
1671800,14571579,"[739876, 870569, 898561, 1694442, 1607245, 239..."
1671801,14571580,"[202353, 888228, 1627186, 1298436, 168977, 155..."


In [35]:
from collections import defaultdict
import pickle

import numpy as np
import pandas as pd
import polars as pl
from annoy import AnnoyIndex
from gensim.models import Word2Vec

In [37]:
train = pl.read_parquet("../input/otto-full-optimized-memory-footprint/train.parquet")
test = pl.read_parquet("../input/otto-full-optimized-memory-footprint/test.parquet")

In [38]:
train.head()

session,aid,ts,type
i32,i32,i32,u8
0,1517085,1659304800,0
0,1563459,1659304904,0
0,1309446,1659367439,0
0,16246,1659367719,0
0,1781822,1659367871,0


In [39]:
sentences_df = pl.concat([train, test]).groupby("session").agg(pl.col("aid").alias("sentence"))

In [40]:
sentences_df.head()

session,sentence
i32,list[i32]
681056,"[1022447, 1022447, ... 1148477]"
14014656,"[1366678, 1698497, ... 360462]"
1721344,"[1827883, 1186885, ... 925595]"
14093472,"[326904, 326904, 1820294]"
13645696,[1605487]


In [41]:
def read_files(path):
    dfs = []
    for file in glob.glob(path):
        df = pd.read_parquet(file)
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

In [43]:
    hoge = read_files("../input/otto-validation/*_parquet/*")

In [44]:
hoge.head()

,session,aid,ts,type
0,12719662,1628069,1661685778000,clicks
1,12719663,523234,1661685778000,clicks
2,12719663,1316756,1661686529000,clicks
3,12719663,424606,1661686795000,clicks
4,12719663,1316756,1661686808000,clicks


In [46]:
fuga = hoge.groupby("session")["aid"].apply(list)

In [48]:
hoge["sentence"] = fuga

In [ ]:
hoge[hoge["session"] == 681056]

In [1]:
from collections import defaultdict
import glob
import pickle

import numpy as np
import pandas as pd
from annoy import AnnoyIndex
from gensim.models import Word2Vec


def dump_pickle(path, o):
    with open(path, "wb") as f:
        pickle.dump(o, f)


def read_files(path):
    dfs = []
    for file in glob.glob(path):
        df = pd.read_parquet(file)
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

In [2]:
train_file_path = "../input/otto-validation/*_parquet/*"
test_file_path = "../input/otto-validation/test_parquet/*"

In [3]:
train = read_files(train_file_path)
train["sentence"] = train.groupby("session")["aid"].apply(list)
test = read_files(test_file_path)
sentences = train["sentence"].to_list()

In [5]:
type(sentences)

list

In [6]:
sentences[0]

[1517085,
 1563459,
 1309446,
 16246,
 1781822,
 1152674,
 1649869,
 461689,
 305831,
 461689,
 362233,
 1649869,
 1649869,
 984597,
 1649869,
 803544,
 1110941,
 1190046,
 1760685,
 631008,
 461689,
 1190046,
 1650637,
 313546,
 1650637,
 979517,
 351157,
 1062149,
 1157384,
 1841388,
 1469630,
 305831,
 1110548,
 1110548,
 305831,
 1650114,
 1604396,
 1009750,
 1800933,
 495779,
 394655,
 495779,
 789245,
 789245,
 366890,
 361317,
 1700164,
 1755597,
 789245,
 784978,
 1171505,
 784978,
 1700164,
 784978,
 1521766,
 1725503,
 528847,
 1816325,
 984597,
 1072782,
 173702,
 1072782,
 1407538,
 1629651,
 1768568,
 1318324,
 1840418,
 1813509,
 1813509,
 667924,
 1226444,
 709550,
 709417,
 1225559,
 1048044,
 1052813,
 1225559,
 240346,
 1582117,
 1707783,
 1624436,
 1157411,
 358305,
 1202970,
 832192,
 1498443,
 723931,
 1436439,
 1693461,
 1206554,
 1110741,
 346352,
 1802050,
 154930,
 964169,
 964169,
 823637,
 964169,
 1411683,
 964169,
 1167722,
 964169,
 1619737,
 964169,
 1840